## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.19,168.86,46.99,96,99,1.01,overcast clouds
1,1,Alta Floresta,BR,-9.88,-56.09,91.65,43,100,2.64,overcast clouds
2,2,Nizhniy Odes,RU,63.64,54.86,21.52,98,90,0.49,overcast clouds
3,3,Cidreira,BR,-30.18,-50.21,70.41,84,3,22.32,clear sky
4,4,Gat,IL,31.61,34.76,77.00,78,20,8.05,few clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum & maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Gat,IL,31.61,34.76,77.00,78,20,8.05,few clouds
5,5,Souillac,MU,-20.52,57.52,75.20,73,75,14.99,shower rain
12,12,Vallenar,CL,-28.57,-70.76,83.53,14,0,9.24,clear sky
22,22,Corn Island,NI,12.17,-83.04,83.35,74,96,6.26,overcast clouds
28,28,Denpasar,ID,-8.65,115.22,78.80,94,20,2.24,few clouds
33,33,Hilo,US,19.73,-155.09,82.40,74,40,5.82,scattered clouds
38,38,Avarua,CK,-21.21,-159.78,75.20,73,49,11.41,light rain
39,39,Namatanai,PG,-3.67,152.43,79.20,82,97,2.71,moderate rain
46,46,Kapaa,US,22.08,-159.32,84.00,83,40,3.62,scattered clouds
54,54,Gamba,GA,-2.65,10.00,76.93,88,100,7.65,light rain


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Gat,IL,31.61,34.76,77.00,78,20,8.05,few clouds
5,5,Souillac,MU,-20.52,57.52,75.20,73,75,14.99,shower rain
12,12,Vallenar,CL,-28.57,-70.76,83.53,14,0,9.24,clear sky
22,22,Corn Island,NI,12.17,-83.04,83.35,74,96,6.26,overcast clouds
28,28,Denpasar,ID,-8.65,115.22,78.80,94,20,2.24,few clouds
33,33,Hilo,US,19.73,-155.09,82.40,74,40,5.82,scattered clouds
38,38,Avarua,CK,-21.21,-159.78,75.20,73,49,11.41,light rain
39,39,Namatanai,PG,-3.67,152.43,79.20,82,97,2.71,moderate rain
46,46,Kapaa,US,22.08,-159.32,84.00,83,40,3.62,scattered clouds
54,54,Gamba,GA,-2.65,10.00,76.93,88,100,7.65,light rain


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Gat,IL,77.00,few clouds,31.61,34.76,
5,Souillac,MU,75.20,shower rain,-20.52,57.52,
12,Vallenar,CL,83.53,clear sky,-28.57,-70.76,
22,Corn Island,NI,83.35,overcast clouds,12.17,-83.04,
28,Denpasar,ID,78.80,few clouds,-8.65,115.22,
33,Hilo,US,82.40,scattered clouds,19.73,-155.09,
38,Avarua,CK,75.20,light rain,-21.21,-159.78,
39,Namatanai,PG,79.20,moderate rain,-3.67,152.43,
46,Kapaa,US,84.00,scattered clouds,22.08,-159.32,
54,Gamba,GA,76.93,light rain,-2.65,10.00,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
print("--------------------------------")
print("Data Retrieval Complete         ")
print("--------------------------------")
         

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
--------------------------------
Data Retrieval Complete         
--------------------------------


In [35]:
import numpy as np

In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace =True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [37]:
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Gat,IL,77.00,few clouds,31.61,34.76,OlusHome
5,Souillac,MU,75.20,shower rain,-20.52,57.52,Bils' Villa
12,Vallenar,CL,83.53,clear sky,-28.57,-70.76,Humacao Bed & Breakfast
22,Corn Island,NI,83.35,overcast clouds,12.17,-83.04,Big Fish Guest House
28,Denpasar,ID,78.80,few clouds,-8.65,115.22,ASTON Denpasar Hotel & Convention Center
33,Hilo,US,82.40,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
38,Avarua,CK,75.20,light rain,-21.21,-159.78,Paradise Inn
39,Namatanai,PG,79.20,moderate rain,-3.67,152.43,Namatanai Hotel
46,Kapaa,US,84.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
56,Hithadhoo,MV,82.98,scattered clouds,-0.60,73.08,Scoop Guest House


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))